In [21]:
from googleapiclient.discovery import build 
import googleapiclient.errors

import pandas as pd

In [30]:
#Declare variables
api_key = ""

channel_ids = [#'UCMUnInmOkrWN4gof9KlhNmQ', #laogao (hidden subscriber)
               'UC0SJCeL0QG4BH_KQALjQE-Q', #bbk
               'UCijVIIfFzspulKc7yWA2Qhg', #friends
              ]

# Get credentials and create an API client
api_service_name = "youtube"
api_version = "v3"
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=api_key)

In [33]:
def get_channel_stats(youtube, channel_ids):
    full_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id= ','.join(channel_ids)
    )
    response = request.execute()
    
#     print(response)

    # loop in the json (response)
    for item in response['items']:
        data = {
            'channelName': item['snippet']['title'],
            'subscribers': item['statistics']['subscriberCount'],
            'views': item['statistics']['viewCount'],
            'totalViews': item['statistics']['videoCount'],
            'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        
        full_data.append(data)
        
    return(pd.DataFrame(full_data))

In [34]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [35]:
channel_stats

,channelName,subscribers,views,totalViews,playlistId
0,Friends,100000,47815561,219,UUijVIIfFzspulKc7yWA2Qhg
1,BBK Network,226000,43322413,307,UU0SJCeL0QG4BH_KQALjQE-Q


In [48]:
def get_video_ids(youtube, playlist_id):
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()

#     print(response)

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    # keep looping until he last page since each time max is only 50
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet, contentDetails",
            playlistId=playlist_id,
            maxResults = 50,
            pageToken = next_page_token
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        
    return video_ids

In [49]:
playlist_id = "UU0SJCeL0QG4BH_KQALjQE-Q"
video_ids = get_video_ids(youtube, playlist_id)

In [52]:
len(video_ids)

307

In [55]:
def get_video_details(youtube, video_ids):

    all_video_info = []

    for i in range(0,len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet, contentDetails, statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats = {
                'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                'contentDetails': ['duration', 'defination','caption']
            }

            video_info = {}
            video_info['video_id'] = video['id']

            for key in stats.keys():
                for value in stats[key]:
                    try:
                        video_info[value] = video[key][value]
                    except:
                        video_info[value] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

In [56]:
video_df = get_video_details(youtube, video_ids)
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,defination,caption
0,pbuyj3sIaK4,BBK Network,鸡鸡都去哪儿了？居然有“幕后黑手”操控鸡肉市场？【新闻随便看25/5】,每天都在等鸡，鸡鸡复鸡鸡，鸡鸡不再多。\n鸡农皆被明日累，明日到来也无鸡。\n\nhttps...,[新闻随便看],2022-05-25T11:00:14Z,67885,3183,None,259,PT10M22S,None,true
1,iLzKHutDyJk,BBK Network,嘲笑了蚊型脚车，然后呢？大马城市化如何侵占乡村与青年休闲区？【议题爆爆看】,一个人出错也许是他犯贱，但一样的错事重复发生，那就是结构问题了\n让我们借由沈可婷案来探讨大...,"[蚊型脚车, 沈可婷]",2022-05-22T11:00:18Z,128120,3937,None,434,PT10M39S,None,true
2,Q5TGRabJ4EA,BBK Network,Coser不可以这样穿？我们就快买不起Roti Canai？【新闻随便看21/5】,二次元被现实暴击？大马会买不到面粉吗？\n\nhttps://bbknetwork.co/ ...,None,2022-05-21T11:00:12Z,124635,3749,None,440,PT13M27S,None,false
3,ASqwkNyRiBc,BBK Network,Guru来踢馆！怎样辨别真假Guru？最犀利直接的提问！ft @陈剑老师投资教育频道,陈剑老师又来访了。投资Guru 还敢来找Lucas？\n谢谢陈剑的出席与分享！\n陈剑的脸书...,[新闻随便看],2022-05-19T11:00:45Z,66995,2137,None,238,PT13M47S,None,false
4,35-G2KnQ1d8,BBK Network,首相要找马斯克来建Tesla厂？魏家祥禁止微型交通工具！【新闻随便看18/5】,马来文居然大放光彩？交通部博位逞英雄？\n\nBossboleh 是一站式线上公司秘书网站\...,None,2022-05-18T11:00:16Z,139374,4065,None,357,PT15M5S,None,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,qA2WUFlGv40,BBK Network,TNB国能乱涨电费？来听听rapper的看法吧！,是的，你没看错，你爷爷订阅的yt复活了。\n今天，卢卡斯很开心被geniuus邀请上来和大家...,"[TNB, Tenaga Nasional Berhad, 国能, 国家能源, 电费涨价, ...",2020-06-24T11:00:13Z,7778,351,None,44,PT6M32S,None,true
303,ZCN_t5K4X-c,BBK Network,拉曼生大爆料！马华和行动党斗争，谁害惨学生？政治开箱Ep04,是的，今天《政治开箱》终于回来了！\n\n这次，我们要去深入探讨为什么政府停止拨款给学生，又...,"[拉曼拨款, 马华, 行动党, 补选, 林冠英, 魏家祥, 爆爆看, 新闻爆爆看, 卢卡斯,...",2019-11-12T10:00:16Z,119305,1957,None,330,PT17M40S,None,false
304,-_L8TGhPUHw,BBK Network,张念群如何应对统考华教等议题？来听副教育部长的教育理念！- 政治开箱03,教育部，作为牵涉所有国民的领域，是内阁当中受到最大关注度的部门之一。尤其在固有“再穷也不能穷...,"[新闻爆爆看, 爆爆看, 卢卡斯, 红药丸, 张念群, 民主行动党, 行动党, 火箭党, 三...",2019-07-12T10:00:01Z,95670,1643,None,293,PT21M38S,None,false
305,OJp9zRjncFs,BBK Network,YB倪可敏为什么发脾气？跟随国会议员走访安顺！- 政治开箱02,今天，本台主播卢卡斯又要来跟大家继续政治开箱，一起来看看新政府的运作与挑战！\n\n这一集呢...,"[倪可敏, 希望联盟, 国会议员, 安顺, 卢卡斯, 红药丸, 大马政治, 509一周年, ...",2019-05-03T10:00:03Z,341519,6151,None,412,PT17M,None,false
